## Solving the environment

- ALSO: this might be a great example to try supervised learning and show why it doesn't work...???
  - in some ways this is a way better example than frozen lake... because there IS a short-term reward, it's just not what you should look at.
  - with frozen lake there is no short-term reward, so RL seems "obvious"


- REWARD SHAPING 2 types:
  - help it learn
  - actually change what you care about
- examples here: ethical recommendations
- some sort of total reward for the whole episode, not interim rewards
- other stuff